In [1]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [2]:
ws = Workspace.from_config()
ws

Workspace.create(name='resnet', subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e', resource_group='azureml_uw_resnet')

In [3]:
cluster_name = "ML-CPU-test"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('No compute cluster named {}'.format(cluster_name))
    exit()

Found existing compute target


In [4]:
curated_env_name = 'Resnet50v15-CPU-cluster'
#pytorch_env = Environment.get(workspace=ws, name=curated_env_name)
pytorch_env = Environment.from_conda_specification(name=curated_env_name, file_path='./conda_dependencies.yml')

In [30]:
from azureml.core import ScriptRunConfig
from azureml.core.datastore import Datastore
from azureml.core import Dataset

project_folder = './'
data_path = 'datasets'

datastore = Datastore.get(ws, 'workspaceblobstore')
dataset = Dataset.File.from_files(path=(datastore, data_path))
data_loc = dataset.as_named_input('input').as_mount()
print(data_loc)

In [31]:
src = ScriptRunConfig(source_directory=project_folder,
                        # command=['ls'],
                        script='train_resnet.py',
                        arguments=[
                          '--num_epochs', 16,
                          '--batch', '32',
                          '--shuffle', 'True',
                          '--dataloc', data_loc,
                          '--output_dir', './outputs',
                        ],
                        compute_target=compute_target,
                        environment=pytorch_env)

run = Experiment(ws, name='Train-Resnet50v15').submit(src)
run.wait_for_completion(show_output=True)

RunId: Train-Resnet50v15_1622607071_8955f637
Web View: https://ml.azure.com/runs/Train-Resnet50v15_1622607071_8955f637?wsid=/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourcegroups/azureml_uw_resnet/workspaces/resnet&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_fca8b318714dc083092be9e4cfc80123ee8f881eda77258a898cc7b5df6abfd7_p.txt

2021-06-02T04:11:36Z Successfully mounted a/an Blobfuse File System at /mnt/batch/tasks/shared/LS_root/jobs/resnet/azureml/train-resnet50v15_1622607071_8955f637/mounts/workspaceblobstore
2021-06-02T04:11:37Z Starting output-watcher...
2021-06-02T04:11:37Z IsDedicatedCompute == False, starting polling for Low-Pri Preemption
2021-06-02T04:11:37Z Executing 'Copy ACR Details file' on 10.0.0.4
2021-06-02T04:11:38Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_fe4afc798de401edfb76dc27a38b1703
Digest: sha256:5